<a href="https://colab.research.google.com/github/rafabandoni/world-cup-22-predict/blob/main/predict_worldcup22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

teste

Removing warnings from deprecated functions

In [ ]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

Importing libraries

In [ ]:
from utils.enriching_data import enrich_data
from utils.run_predict import run_predict_home
from utils.run_predict import run_predict_away

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import numpy as np
import pandas as pd
import random

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

# Defining Functions

# Read Datasets

Historical results dataset

In [ ]:
historical_results = pd.read_csv('https://raw.githubusercontent.com/rafabandoni/world-cup-22-predict/main/data/historical-results.csv')
historical_results.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


In [ ]:
historical_results = historical_results.replace('United States','USA', regex=True) # changing for pattern

Win, loose and draw ratio dataset

In [ ]:
historical_win_loose_draw_ratios = pd.read_csv('https://raw.githubusercontent.com/rafabandoni/world-cup-22-predict/main/data/historical_win-loose-draw_ratios.csv')
historical_win_loose_draw_ratios.head(5)

,country1,country2,games,wins,looses,draws
0,Argentina,Australia,7,0.714286,0.142857,0.142857
1,Australia,Argentina,7,0.142857,0.714286,0.142857
2,Argentina,Belgium,4,0.750000,0.250000,0.000000
3,Belgium,Argentina,4,0.250000,0.750000,0.000000
4,Argentina,Brazil,108,0.361111,0.398148,0.240741


FIFA ranking dataset

In [ ]:
ranking = pd.read_csv('https://raw.githubusercontent.com/rafabandoni/world-cup-22-predict/main/data/ranking.csv')
ranking.head(5)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
1,96,Syria,SYR,11.0,0.0,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.0,0.0,0,CAF,1992-12-31
3,99,Latvia,LVA,10.0,0.0,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.0,0.0,0,CAF,1992-12-31


Game shootouts historic dataset

In [ ]:
shootouts = pd.read_csv('https://raw.githubusercontent.com/rafabandoni/world-cup-22-predict/main/data/shootouts.csv')
shootouts.head(5)

,date,home_team,away_team,winner
0,1967-08-22,India,Taiwan,Taiwan
1,1971-11-14,South Korea,Vietnam Republic,South Korea
2,1972-05-17,Thailand,South Korea,South Korea
3,1972-05-19,Thailand,Cambodia,Thailand
4,1973-04-21,Senegal,Ghana,Ghana


# Treating Historical Results Dataset

Checking data

In [ ]:
historical_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


In [ ]:
historical_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44060 entries, 0 to 44059
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        44060 non-null  object 
 1   home_team   44060 non-null  object 
 2   away_team   44060 non-null  object 
 3   home_score  44059 non-null  float64
 4   away_score  44059 non-null  float64
 5   tournament  44060 non-null  object 
 6   city        44060 non-null  object 
 7   country     44060 non-null  object 
 8   neutral     44060 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB


Top 10 tournaments on database:

In [ ]:
historical_results.groupby('tournament').count().sort_values('date', ascending=False)['date'].head(10)

tournament
Friendly                                17425
FIFA World Cup qualification             7774
UEFA Euro qualification                  2593
African Cup of Nations qualification     1932
FIFA World Cup                            900
Copa América                              841
AFC Asian Cup qualification               764
African Cup of Nations                    742
CECAFA Cup                                620
CFU Caribbean Cup qualification           606
Name: date, dtype: int64

## Aggregations

Filtering to only world cup data, adding Qatar and Wales data since there's nothing about them on world cup data

In [ ]:
historical_results['date'] = historical_results['date'].astype('datetime64') # fixing date format

# using just fifa tournaments
historical_results_world_cup = historical_results.query('tournament.str.contains("FIFA")').sort_values('date').reset_index(drop=True)

historical_results_world_cup.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1930-07-13,Belgium,USA,0.0,3.0,FIFA World Cup,Montevideo,Uruguay,True
1,1930-07-13,France,Mexico,4.0,1.0,FIFA World Cup,Montevideo,Uruguay,True
2,1930-07-14,Brazil,Yugoslavia,1.0,2.0,FIFA World Cup,Montevideo,Uruguay,True
3,1930-07-14,Peru,Romania,1.0,3.0,FIFA World Cup,Montevideo,Uruguay,True
4,1930-07-15,Argentina,France,1.0,0.0,FIFA World Cup,Montevideo,Uruguay,True
5,1930-07-16,Chile,Mexico,3.0,0.0,FIFA World Cup,Montevideo,Uruguay,True
6,1930-07-17,Bolivia,Yugoslavia,0.0,4.0,FIFA World Cup,Montevideo,Uruguay,True
7,1930-07-17,Paraguay,USA,0.0,3.0,FIFA World Cup,Montevideo,Uruguay,True
8,1930-07-18,Uruguay,Peru,1.0,0.0,FIFA World Cup,Montevideo,Uruguay,False
9,1930-07-19,Argentina,Mexico,6.0,3.0,FIFA World Cup,Montevideo,Uruguay,True


Identifying finals

In [ ]:
historical_results_world_cup['is_final'] = ''

for i in range(len(historical_results_world_cup['date'])):
  if i < 899 and historical_results_world_cup[i:i+1]['date'].astype(str).str[:4][i] != historical_results_world_cup[i+1:i+2]['date'].astype(str).str[:4][i+1]:
    historical_results_world_cup['is_final'][i] = 'TRUE'
  elif i < 899:
    historical_results_world_cup['is_final'][i] = 'FALSE'
  elif i == 899:
    historical_results_world_cup['is_final'][i] = 'TRUE'

# Historical Results with possibily to win

Addying winner and loosers

In [ ]:
historical_results_world_cup['winner'] = ''

for i in range(len(historical_results_world_cup['date'])):
  if historical_results_world_cup['home_score'][i] > historical_results_world_cup['away_score'][i]:
    historical_results_world_cup['winner'][i] = 'home_win'
  elif historical_results_world_cup['home_score'][i] < historical_results_world_cup['away_score'][i]:
    historical_results_world_cup['winner'][i] = 'away_win'
  else:
    historical_results_world_cup['winner'][i] = 'draw'

In [ ]:
# adding winners and loosers
historical_results_world_cup = historical_results_world_cup.merge(historical_win_loose_draw_ratios, left_on=['home_team', 'away_team'], right_on=['country1', 'country2'], how='left').rename(columns={
    'wins' : 'home_wins',
    'looses' : 'home_looses'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score', 'tournament', 'city', 'country', 'neutral', 'is_final', 'winner', 'games', 'home_wins', 'home_looses', 'draws']]

historical_results_world_cup = historical_results_world_cup.replace(np.nan, 0) # adding for teams that never played before

# Historical result with ranking on date

Checking data

In [ ]:
ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63916 entries, 0 to 63915
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rank             63916 non-null  int64  
 1   country_full     63916 non-null  object 
 2   country_abrv     63916 non-null  object 
 3   total_points     63916 non-null  float64
 4   previous_points  63916 non-null  float64
 5   rank_change      63916 non-null  int64  
 6   confederation    63916 non-null  object 
 7   rank_date        63916 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 3.9+ MB


Adding home and away rank

In [ ]:
# home rank
historical_results_world_cup = historical_results_world_cup.merge(ranking, left_on=[historical_results_world_cup['date'].astype(str).str[:7], 'home_team'],
                                            right_on=[ranking['rank_date'].str[:7], 'country_full'],
                                            how='left').rename(columns={
                                                'rank' : 'home_rank',
                                                'total_points' : 'home_ranking_points'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score',
    'tournament', 'city', 'country', 'neutral', 'is_final', 'winner',
    'games', 'home_wins', 'home_looses', 'draws', 'home_rank', 'home_ranking_points']]

# away rank
historical_results_world_cup = historical_results_world_cup.merge(ranking, left_on=[historical_results_world_cup['date'].astype(str).str[:7], 'away_team'],
                                            right_on=[ranking['rank_date'].str[:7], 'country_full'],
                                            how='left').rename(columns={
                                                'rank' : 'away_rank',
                                                'total_points' : 'away_ranking_points'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score',
    'tournament', 'city', 'country', 'neutral', 'is_final', 'winner',
    'games', 'home_wins', 'home_looses', 'draws', 'home_rank', 'home_ranking_points',
    'away_rank', 'away_ranking_points']]

In [ ]:
historical_results_world_cup

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,is_final,winner,games,home_wins,home_looses,draws,home_rank,home_ranking_points,away_rank,away_ranking_points
0,1930-07-13,Belgium,USA,0.0,3.0,FIFA World Cup,Montevideo,Uruguay,True,FALSE,away_win,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
1,1930-07-13,France,Mexico,4.0,1.0,FIFA World Cup,Montevideo,Uruguay,True,FALSE,home_win,7.0,0.714286,0.142857,0.142857,NaN,NaN,NaN,NaN
2,1930-07-14,Brazil,Yugoslavia,1.0,2.0,FIFA World Cup,Montevideo,Uruguay,True,FALSE,away_win,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
3,1930-07-14,Peru,Romania,1.0,3.0,FIFA World Cup,Montevideo,Uruguay,True,FALSE,away_win,0.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
4,1930-07-15,Argentina,France,1.0,0.0,FIFA World Cup,Montevideo,Uruguay,True,FALSE,home_win,12.0,0.500000,0.250000,0.250000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8670,2022-03-30,Solomon Islands,New Zealand,0.0,5.0,FIFA World Cup qualification,Doha,Qatar,True,,away_win,0.0,0.000000,0.000000,0.000000,137.0,1092.56,101.0,1206.07
8671,2022-06-01,Scotland,Ukraine,1.0,3.0,FIFA World Cup qualification,Glasgow,Scotland,False,,away_win,0.0,0.000000,0.000000,0.000000,45.0,1462.96,27.0,1542.79
8672,2022-06-05,Wales,Ukraine,1.0,0.0,FIFA World Cup qualification,Cardiff,Wales,False,,home_win,0.0,0.000000,0.000000,0.000000,19.0,1582.13,27.0,1542.79
8673,2022-06-13,Australia,Peru,0.0,0.0,FIFA World Cup qualification,Al Rayyan,Qatar,True,,draw,0.0,0.000000,0.000000,0.000000,39.0,1483.73,21.0,1562.24


# Final historical dataset

In [ ]:
historical_results_world_cup = historical_results_world_cup.replace(np.nan, 0) # replacing null ranks

# fixing data types
historical_results_world_cup['home_rank'] = historical_results_world_cup['home_rank'].astype('float64')
historical_results_world_cup['home_ranking_points'] = historical_results_world_cup['home_ranking_points'].astype('float64')
historical_results_world_cup['away_rank'] = historical_results_world_cup['away_rank'].astype('float64')
historical_results_world_cup['away_ranking_points'] = historical_results_world_cup['away_ranking_points'].astype('float64')

# Algoritmo

Feature engineering

In [ ]:
# one-hot encode the data using pandas get_dummies
features = pd.get_dummies(historical_results_world_cup)

# labels are the values we want to predict
train_labels = np.array(features.query('date < 2018')['home_score'])

# remove the labels from the features
# axis 1 refers to the columns
train_features = features.query('date < 2018').drop('home_score', axis = 1).drop('date', axis = 1)
# convert to numpy array
train_features = np.array(train_features)

# creating labels and features
test_labels = np.array(features.query('date >= 2018')['home_score'])
test_features = features.query('date >= 2018').drop('home_score', axis = 1).drop('date', axis = 1)
test_features = np.array(test_features)

# checking labels and features
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (7748, 1466)
Training Labels Shape: (7748,)
Testing Features Shape: (927, 1466)
Testing Labels Shape: (927,)


Running train and test

In [ ]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators=1000, random_state=42)

# Train the model on training data
rf.fit(train_features, train_labels)

# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.77 degrees.


In [ ]:
# merging test data and predicts data
df_predict_test = pd.merge(pd.DataFrame(predictions.round(0)), pd.DataFrame(test_labels), left_index=True, right_index=True).rename(columns={'0_x' : 'predicts', '0_y' : 'reality'})
df_predict_test['is_correct'] = df_predict_test['predicts'] - df_predict_test['reality']
predict_right = ((df_predict_test['reality'].count() - df_predict_test.query('is_correct != 0')['is_correct'].count()) / df_predict_test['reality'].count() * 100).round(2)

print(f"The algorithm predicted right: {predict_right}% of the values")

The algorithm predicted right: 55.56% of the values


# 2022 calendar predictions

Reading and treating data

In [ ]:
matches_schedule = pd.read_csv('https://raw.githubusercontent.com/rafabandoni/world-cup-22-predict/main/data/matches-schedule.csv')

# fixing date format
matches_schedule['date'] = matches_schedule['date'].str[-4:] + '-' + matches_schedule['date'].str[3:5] + '-' + matches_schedule['date'].str[:2]

# adding score columns
matches_schedule['home_score'] = 0
matches_schedule['away_score'] = 0

# renaming for pattern
matches_schedule.rename(columns={'country1' : 'home_team', 'country2' : 'away_team'}, inplace=True)
matches_schedule = matches_schedule[['date', 'home_team', 'home_score', 'away_score', 'away_team', 'phase']]

# fixing datatype for date
matches_schedule['date'] = matches_schedule['date'].astype('datetime64')

Adding historical ratio and rank

In [ ]:
# adding outcome ratio
matches_schedule = matches_schedule.merge(historical_win_loose_draw_ratios, left_on=['home_team', 'away_team'], right_on=['country1', 'country2'], how='left').rename(columns={
    'wins' : 'home_wins',
    'looses' : 'home_looses'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score', 'phase', 'games', 'home_wins', 'home_looses', 'draws']]

# adding home rank
matches_schedule = matches_schedule.merge(ranking.query('rank_date.astype("datetime64") > 2021').groupby('country_full').mean().round(0),
                                          left_on='home_team',
                                          right_on='country_full',
                                          how='left').rename(columns={
                                                                      'rank' : 'home_rank',
                                                                      'total_points' : 'home_ranking_points'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score',
    'phase', 'games', 'home_wins', 'home_looses', 'draws', 'home_rank', 'home_ranking_points']]

# adding away rank
matches_schedule = matches_schedule.merge(ranking.query('rank_date.astype("datetime64") > 2021').groupby('country_full').mean().round(0),
                                          left_on='away_team',
                                          right_on='country_full',
                                          how='left').rename(columns={
                                                                      'rank' : 'away_rank',
                                                                      'total_points' : 'away_ranking_points'
})[['date', 'home_team', 'away_team', 'home_score', 'away_score',
    'phase', 'games', 'home_wins', 'home_looses', 'draws', 'home_rank',
    'home_ranking_points', 'away_rank', 'away_ranking_points']]

In [ ]:
matches_schedule = matches_schedule.fillna(0) # to fix countries with no historical data

## Running algorithm

### Group Stage

In [ ]:
matches_schedule

,date,home_team,away_team,home_score,away_score,phase,games,home_wins,home_looses,draws,home_rank,home_ranking_points,away_rank,away_ranking_points
0,2022-11-21,Qatar,Ecuador,0,0,group matches,3.0,0.333333,0.333333,0.333333,50.0,1431.0,49.0,1436.0
1,2022-11-21,Senegal,Netherlands,0,0,group matches,0.0,0.000000,0.000000,0.000000,20.0,1567.0,11.0,1647.0
2,2022-11-21,England,Iran,0,0,group matches,0.0,0.000000,0.000000,0.000000,4.0,1733.0,0.0,0.0
3,2022-11-21,USA,Wales,0,0,group matches,0.0,0.000000,0.000000,0.000000,15.0,1620.0,19.0,1574.0
4,2022-11-22,France,Australia,0,0,group matches,5.0,0.600000,0.200000,0.200000,3.0,1769.0,38.0,1478.0
5,2022-11-22,Denmark,Tunisia,0,0,group matches,1.0,1.000000,0.000000,0.000000,10.0,1651.0,29.0,1510.0
6,2022-11-22,Mexico,Poland,0,0,group matches,9.0,0.333333,0.333333,0.333333,11.0,1647.0,25.0,1540.0
7,2022-11-22,Argentina,Saudi Arabia,0,0,group matches,4.0,0.500000,0.000000,0.500000,5.0,1727.0,55.0,1413.0
8,2022-11-23,Belgium,Canada,0,0,group matches,1.0,1.000000,0.000000,0.000000,1.0,1816.0,50.0,1430.0
9,2022-11-23,Spain,Costa Rica,0,0,group matches,3.0,0.666667,0.000000,0.333333,7.0,1689.0,43.0,1457.0


Home prediction:

In [ ]:
# one-hot encode the data using pandas get_dummies
features = pd.get_dummies(
    pd.concat([historical_results_world_cup,matches_schedule]).reset_index(drop=True)
    [['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'games', 'home_wins', 'home_looses', 'draws', 'home_rank',
       'home_ranking_points', 'away_rank', 'away_ranking_points']]
    )

# labels are the values we want to predict
train_labels = np.array(features.query('date < "2022-11-01"')['home_score'])

# remove the labels from the features
# axis 1 refers to the columns
train_features = features.query('date < "2022-11-01"').drop('home_score', axis = 1).drop('date', axis = 1)
# convert to numpy array
train_features = np.array(train_features)

test_labels = np.array(features.query('date >= "2022-11-01"')['home_score'])
test_features = features.query('date >= "2022-11-01"').drop('home_score', axis = 1).drop('date', axis = 1)
test_features = np.array(test_features)

# instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# train the model on training data
rf.fit(train_features, train_labels)

# use the forest's predict method on the test data
predictions = rf.predict(test_features)

#adding predicts to df
matches_schedule_predicted = pd.merge(pd.DataFrame(predictions.round(0)), matches_schedule, left_index=True, right_index=True)
matches_schedule_predicted.rename(columns={0 : "home_score_predicted"}, inplace=True)
matches_schedule_predicted = matches_schedule_predicted[['date', 'home_team', 'home_score_predicted', 'away_score', 'away_team']]

Away prediction:

In [ ]:
# one-hot encode the data using pandas get_dummies
features = pd.get_dummies(
    pd.concat([historical_results_world_cup,matches_schedule]).reset_index(drop=True)
    [['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'games', 'home_wins', 'home_looses', 'draws', 'home_rank',
       'home_ranking_points', 'away_rank', 'away_ranking_points']]
    )

# labels are the values we want to predict
train_labels = np.array(features.query('date < "2022-11-01"')['away_score'])

# remove the labels from the features
# axis 1 refers to the columns
train_features = features.query('date < "2022-11-01"').drop('away_score', axis = 1).drop('date', axis = 1)
# convert to numpy array
train_features = np.array(train_features)

test_labels = np.array(features.query('date >= "2022-11-01"')['away_score'])
test_features = features.query('date >= "2022-11-01"').drop('away_score', axis = 1).drop('date', axis = 1)
test_features = np.array(test_features)

# instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# train the model on training data
rf.fit(train_features, train_labels)

# use the forest's predict method on the test data
predictions = rf.predict(test_features)

#adding predicts to df
matches_schedule_predicted = pd.merge(pd.DataFrame(predictions.round(0)), matches_schedule_predicted, left_index=True, right_index=True)
matches_schedule_predicted.rename(columns={0 : "away_score_predicted"}, inplace=True)
matches_schedule_predicted = matches_schedule_predicted[['date', 'home_team', 'home_score_predicted', 'away_score_predicted', 'away_team']]

Checking predicts

In [ ]:
matches_schedule_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team
0,2022-11-21,Qatar,2.0,1.0,Ecuador
1,2022-11-21,Senegal,1.0,1.0,Netherlands
2,2022-11-21,England,3.0,1.0,Iran
3,2022-11-21,USA,1.0,1.0,Wales
4,2022-11-22,France,2.0,1.0,Australia
5,2022-11-22,Denmark,2.0,1.0,Tunisia
6,2022-11-22,Mexico,1.0,1.0,Poland
7,2022-11-22,Argentina,1.0,1.0,Saudi Arabia
8,2022-11-23,Belgium,3.0,1.0,Canada
9,2022-11-23,Spain,2.0,1.0,Costa Rica


Exporting CSV

In [ ]:
matches_schedule_predicted.to_csv('predicts/matches_schedule_predicted.csv', index=False)

## Predicting next phases based on algorithm predictions

Returning winners

In [ ]:
matches_schedule_predicted['result'] = ''

for i in range(0, matches_schedule_predicted['home_team'].count()):
    if matches_schedule_predicted['home_score_predicted'][i] > matches_schedule_predicted['away_score_predicted'][i]:
        matches_schedule_predicted['result'][i] = 'home_win'
    elif matches_schedule_predicted['home_score_predicted'][i] < matches_schedule_predicted['away_score_predicted'][i]:
        matches_schedule_predicted['result'][i] = 'away_win'
    else:
        matches_schedule_predicted['result'][i] = 'draw'

Returning points based on result

In [ ]:
matches_schedule_predicted['home_points'] = 0
matches_schedule_predicted['away_points'] = 0

for i in range(0, matches_schedule_predicted['result'].count()):
    if matches_schedule_predicted['result'][i] == 'home_win':
        matches_schedule_predicted['home_points'][i] = 3
    elif matches_schedule_predicted['result'][i] == 'away_win':
        matches_schedule_predicted['away_points'][i] = 3
    else:
        matches_schedule_predicted['home_points'][i] = 1
        matches_schedule_predicted['away_points'][i] = 1

In [ ]:
points_as_home = matches_schedule_predicted.groupby('home_team').sum().reset_index(drop=False)[['home_team', 'home_points']]
points_as_home = points_as_home[['home_team', 'home_points']]
points_as_home.rename(columns={'home_team' : 'team'}, inplace=True)

points_as_away = matches_schedule_predicted.groupby('away_team').sum().reset_index(drop=False)[['away_team', 'away_points']]
points_as_away = points_as_away[['away_team', 'away_points']]
points_as_away.rename(columns={'away_team' : 'team'}, inplace=True)

points_df = pd.merge(points_as_home, points_as_away)
points_df['total_points'] = points_df['home_points'] + points_df['away_points']
points_df = points_df[['team', 'total_points']]

Adding groups

In [ ]:
groups = {
    'Qatar' : 'group_a',
    'Senegal' : 'group_a',
    'England' : 'group_b',
    'USA' : 'group_b',
    'France' : 'group_d',
    'Denmark' : 'group_d',
    'Mexico' : 'group_c',
    'Argentina' : 'group_c',
    'Belgium' : 'group_f',
    'Spain' : 'group_e',
    'Germany' : 'group_e',
    'Morocco' : 'group_f',
    'Switzerland' : 'group_g',
    'Uruguay' : 'group_h',
    'Portugal' : 'group_h',
    'Brazil' : 'group_g',
    'Wales' : 'group_b',
    'Netherlands' : 'group_a',
    'Tunisia' : 'group_d',
    'Poland' : 'group_c',
    'Japan' : 'group_e',
    'Croatia' : 'group_f',
    'Cameroon' : 'group_g',
    'South Korea' : 'group_h',
    'Iran' : 'group_b',
    'Ecuador' : 'group_a',
    'Australia' : 'group_d',
    'Saudi Arabia' : 'group_c',
    'Canada' : 'group_f',
    'Costa Rica' : 'group_e',
    'Ghana' : 'group_h',
    'Serbia' : 'group_g'
}

points_df['group'] = points_df['team'].map(groups)

points_df = points_df.sort_values(['group', 'total_points'], ascending=[True, False])

Checking who passes based on first predict

In [ ]:
pass_df = points_df.loc[pd.DataFrame(list(points_df.groupby(['group'])['total_points'].nlargest(2).index))[1]]

pass_df

,team,total_points,group
18,Netherlands,7,group_a
23,Senegal,5,group_a
10,England,9,group_b
29,USA,4,group_b
0,Argentina,5,group_c
19,Poland,5,group_c
11,France,7,group_d
8,Denmark,4,group_d
12,Germany,9,group_e
26,Spain,6,group_e


### Phase of 16

Creating Phase of 16 (First from one group v second from next group)

In [ ]:
# ingesting data
data = [
    ['2022-12-03', 'Netherlands', 0, 0, 'USA', 'stage of 16'],
    ['2022-12-03', 'Argentina', 0, 0, 'Denmark', 'stage of 16'],
    ['2022-12-04', 'France', 0, 0, 'Poland', 'stage of 16'],
    ['2022-12-04', 'England', 0, 0, 'Senegal', 'stage of 16'],
    ['2022-12-05', 'Germany', 0, 0, 'Croatia', 'stage of 16'],
    ['2022-12-05', 'Brazil', 0, 0, 'Portugal', 'stage of 16'],
    ['2022-12-06', 'Belgium', 0, 0, 'Spain', 'stage of 16'],
    ['2022-12-06', 'Uruguay', 0, 0, 'Switzerland', 'stage of 16']
    ]

# creating pandas DataFrame
matches_predicted_phase16 = pd.DataFrame(data, columns=['date', 'home_team', 'home_score', 'away_score', 'away_team', 'phase'])

# checking data
matches_predicted_phase16

,date,home_team,home_score,away_score,away_team,phase
0,2022-12-03,Netherlands,0,0,USA,stage of 16
1,2022-12-03,Argentina,0,0,Denmark,stage of 16
2,2022-12-04,France,0,0,Poland,stage of 16
3,2022-12-04,England,0,0,Senegal,stage of 16
4,2022-12-05,Germany,0,0,Croatia,stage of 16
5,2022-12-05,Brazil,0,0,Portugal,stage of 16
6,2022-12-06,Belgium,0,0,Spain,stage of 16
7,2022-12-06,Uruguay,0,0,Switzerland,stage of 16


Enriching data

In [ ]:
matches_predicted_phase16 = enrich_data(matches_predicted_phase16, historical_win_loose_draw_ratios, ranking)

Home predictions

In [ ]:
matches_predicted_phase16_predicted = run_predict_home(matches_predicted_phase16, historical_results_world_cup)

Away Predictions

In [ ]:
matches_predicted_phase16_predicted = run_predict_away(matches_predicted_phase16, matches_predicted_phase16_predicted, historical_results_world_cup)

Checking predictions

In [ ]:
matches_predicted_phase16_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team
0,2022-12-03,Netherlands,1.0,0.0,USA
1,2022-12-03,Argentina,1.0,1.0,Denmark
2,2022-12-04,France,2.0,1.0,Poland
3,2022-12-04,England,1.0,0.0,Senegal
4,2022-12-05,Germany,2.0,2.0,Croatia
5,2022-12-05,Brazil,2.0,1.0,Portugal
6,2022-12-06,Belgium,1.0,1.0,Spain
7,2022-12-06,Uruguay,1.0,1.0,Switzerland


In [ ]:
matches_predicted_phase16_predicted.to_csv('predicts_stands/phase_of_16_predictions.csv', index=False)

### Phase of 8

Creating Phase of 8

In [ ]:
matches_predicted_phase16_predicted['next_phase'] = 0

for i in range(0, matches_predicted_phase16_predicted['date'].count()):
    if matches_predicted_phase16_predicted['home_score_predicted'][i] > matches_predicted_phase16_predicted['away_score_predicted'][i]:
        matches_predicted_phase16_predicted['next_phase'][i] = 1
    elif matches_predicted_phase16_predicted['home_score_predicted'][i] < matches_predicted_phase16_predicted['away_score_predicted'][i]:
        matches_predicted_phase16_predicted['next_phase'][i] = 2
    else:
        matches_predicted_phase16_predicted['next_phase'][i] = random.randint(1, 2)

matches_predicted_phase16_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team,next_phase
0,2022-12-03,Netherlands,1.0,0.0,USA,1
1,2022-12-03,Argentina,1.0,1.0,Denmark,1
2,2022-12-04,France,2.0,1.0,Poland,1
3,2022-12-04,England,1.0,0.0,Senegal,1
4,2022-12-05,Germany,2.0,2.0,Croatia,1
5,2022-12-05,Brazil,2.0,1.0,Portugal,1
6,2022-12-06,Belgium,1.0,1.0,Spain,1
7,2022-12-06,Uruguay,1.0,1.0,Switzerland,1


In [ ]:
# ingesting data
data = [
    ['2022-12-09', 'Netherlands', 0, 0, 'Denmark', 'stage of 8'],
    ['2022-12-09', 'Brazil', 0, 0, 'Croatia', 'stage of 8'],
    ['2022-12-10', 'Spain', 0, 0, 'Switzerland', 'stage of 8'],
    ['2022-12-10', 'France', 0, 0, 'England', 'stage of 8']
    ]

# creating pandas DataFrame
matches_predicted_phase8 = pd.DataFrame(data, columns=['date', 'home_team', 'home_score', 'away_score', 'away_team', 'phase'])

# checking data
matches_predicted_phase8

,date,home_team,home_score,away_score,away_team,phase
0,2022-12-09,Netherlands,0,0,Denmark,stage of 8
1,2022-12-09,Brazil,0,0,Croatia,stage of 8
2,2022-12-10,Spain,0,0,Switzerland,stage of 8
3,2022-12-10,France,0,0,England,stage of 8


Enriching Data

In [ ]:
matches_predicted_phase8 = enrich_data(matches_predicted_phase8, historical_win_loose_draw_ratios, ranking)

Home predictions

In [ ]:
matches_predicted_phase8_predicted = run_predict_home(matches_predicted_phase8, historical_results_world_cup)

Away predictions

In [ ]:
matches_predicted_phase8_predicted = run_predict_away(matches_predicted_phase8, matches_predicted_phase8_predicted, historical_results_world_cup)

Checking predictions

In [ ]:
matches_predicted_phase8_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team
0,2022-12-09,Netherlands,1.0,1.0,Denmark
1,2022-12-09,Brazil,2.0,1.0,Croatia
2,2022-12-10,Spain,2.0,1.0,Switzerland
3,2022-12-10,France,1.0,1.0,England


In [ ]:
matches_predicted_phase8_predicted.to_csv('predicts_stands/phase_of_8_predictions.csv', index=False)

### Semifinals

Checking possible penalties results

In [ ]:
matches_predicted_phase8_predicted['next_phase'] = 0

for i in range(0, matches_predicted_phase8_predicted['date'].count()):
    if matches_predicted_phase8_predicted['home_score_predicted'][i] > matches_predicted_phase8_predicted['away_score_predicted'][i]:
        matches_predicted_phase8_predicted['next_phase'][i] = 1
    elif matches_predicted_phase8_predicted['home_score_predicted'][i] < matches_predicted_phase8_predicted['away_score_predicted'][i]:
        matches_predicted_phase8_predicted['next_phase'][i] = 2
    else:
        matches_predicted_phase8_predicted['next_phase'][i] = random.randint(1, 2)

matches_predicted_phase8_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team,next_phase
0,2022-12-09,Netherlands,1.0,1.0,Denmark,1
1,2022-12-09,Brazil,2.0,1.0,Croatia,1
2,2022-12-10,Spain,2.0,1.0,Switzerland,1
3,2022-12-10,France,1.0,1.0,England,1


Creating semifinals

In [ ]:
# ingesting data
data = [
    ['2022-12-13', 'Brazil', 0, 0, 'Netherlands', 'semifinals'],
    ['2022-12-14', 'Spain', 0, 0, 'England', 'semifinals']
    ]

# creating pandas DataFrame
matches_predicted_semifinals = pd.DataFrame(data, columns=['date', 'home_team', 'home_score', 'away_score', 'away_team', 'phase'])

# checking data
matches_predicted_semifinals

,date,home_team,home_score,away_score,away_team,phase
0,2022-12-13,Brazil,0,0,Netherlands,semifinals
1,2022-12-14,Spain,0,0,England,semifinals


Enriching data

In [ ]:
matches_predicted_semifinals = enrich_data(matches_predicted_semifinals, historical_win_loose_draw_ratios, ranking)

Home predicts

In [ ]:
matches_predicted_semifinals_predicted = run_predict_home(matches_predicted_semifinals, historical_results_world_cup)

Away predicts

In [ ]:
matches_predicted_semifinals_predicted = run_predict_away(matches_predicted_semifinals, matches_predicted_semifinals_predicted, historical_results_world_cup)

Checking predictions

In [ ]:
matches_predicted_semifinals_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team
0,2022-12-13,Brazil,1.0,2.0,Netherlands
1,2022-12-14,Spain,1.0,1.0,England


In [ ]:
matches_predicted_semifinals_predicted.to_csv('predicts_stands/semifinals_predictions.csv', index=False)

### Finals

Simulating penalties

In [ ]:
matches_predicted_semifinals_predicted['next_phase'] = 0

for i in range(0, matches_predicted_semifinals_predicted['date'].count()):
    if matches_predicted_semifinals_predicted['home_score_predicted'][i] > matches_predicted_semifinals_predicted['away_score_predicted'][i]:
        matches_predicted_semifinals_predicted['next_phase'][i] = 1
    elif matches_predicted_semifinals_predicted['home_score_predicted'][i] < matches_predicted_semifinals_predicted['away_score_predicted'][i]:
        matches_predicted_semifinals_predicted['next_phase'][i] = 2
    else:
        matches_predicted_semifinals_predicted['next_phase'][i] = random.randint(1, 2)

matches_predicted_semifinals_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team,next_phase
0,2022-12-13,Brazil,1.0,2.0,Netherlands,2
1,2022-12-14,Spain,1.0,1.0,England,1


Creating finals

In [ ]:
# ingesting data
data = [
    ['2022-12-18', 'Netherlands', 0, 0, 'Spain', 'finals'],
    ['2022-12-17', 'Brazil', 0, 0, 'England', 'third place']
    ]

# creating pandas DataFrame
matches_predicted_finals = pd.DataFrame(data, columns=['date', 'home_team', 'home_score', 'away_score', 'away_team', 'phase'])

# checking data
matches_predicted_finals

,date,home_team,home_score,away_score,away_team,phase
0,2022-12-18,Netherlands,0,0,Spain,finals
1,2022-12-17,Brazil,0,0,England,third place


Enrichment and predicts

In [ ]:
# enrichment
matches_predicted_finals = enrich_data(matches_predicted_finals, historical_win_loose_draw_ratios, ranking)

# home predicts
matches_predicted_finals_predicted = run_predict_home(matches_predicted_finals, historical_results_world_cup)

# away predicts
matches_predicted_finals_predicted = run_predict_away(matches_predicted_finals, matches_predicted_finals_predicted, historical_results_world_cup)

Checking predictions

In [ ]:
matches_predicted_finals_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team
0,2022-12-18,Netherlands,2.0,1.0,Spain
1,2022-12-17,Brazil,1.0,1.0,England


In [ ]:
matches_predicted_finals_predicted.to_csv('predicts_stands/finals_predictions.csv', index=False)

Simulating penalties

In [ ]:
matches_predicted_finals_predicted['next_phase'] = 0

for i in range(0, matches_predicted_finals_predicted['date'].count()):
    if matches_predicted_finals_predicted['home_score_predicted'][i] > matches_predicted_finals_predicted['away_score_predicted'][i]:
        matches_predicted_finals_predicted['next_phase'][i] = 1
    elif matches_predicted_finals_predicted['home_score_predicted'][i] < matches_predicted_finals_predicted['away_score_predicted'][i]:
        matches_predicted_finals_predicted['next_phase'][i] = 2
    else:
        matches_predicted_finals_predicted['next_phase'][i] = random.randint(1, 2)

matches_predicted_finals_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team,next_phase
0,2022-12-18,Netherlands,2.0,1.0,Spain,1
1,2022-12-17,Brazil,1.0,1.0,England,1


## Based on reality

### Playoffs

Phase of 16

In [ ]:
# ingesting data
data = [
    ['2022-12-03', 'Netherlands', 0, 0, 'USA', 'stage of 16'],
    ['2022-12-03', 'Argentina', 0, 0, 'Australia', 'stage of 16'],
    ['2022-12-04', 'France', 0, 0, 'Poland', 'stage of 16'],
    ['2022-12-04', 'England', 0, 0, 'Senegal', 'stage of 16'],
    ['2022-12-05', 'Japan', 0, 0, 'Croatia', 'stage of 16'],
    ['2022-12-05', 'Brazil', 0, 0, 'South Korea', 'stage of 16'],
    ['2022-12-06', 'Morocco', 0, 0, 'Spain', 'stage of 16'],
    ['2022-12-06', 'Portugal', 0, 0, 'Switzerland', 'stage of 16']
    ]

# creating pandas DataFrame
matches_schedule_phase16 = pd.DataFrame(data, columns=['date', 'home_team', 'home_score', 'away_score', 'away_team', 'phase'])

# checking data
matches_schedule_phase16

,date,home_team,home_score,away_score,away_team,phase
0,2022-12-03,Netherlands,0,0,USA,stage of 16
1,2022-12-03,Argentina,0,0,Australia,stage of 16
2,2022-12-04,France,0,0,Poland,stage of 16
3,2022-12-04,England,0,0,Senegal,stage of 16
4,2022-12-05,Japan,0,0,Croatia,stage of 16
5,2022-12-05,Brazil,0,0,South Korea,stage of 16
6,2022-12-06,Morocco,0,0,Spain,stage of 16
7,2022-12-06,Portugal,0,0,Switzerland,stage of 16


Enriching data, predicting home and away

In [ ]:
# enrichment
matches_schedule_phase16 = enrich_data(matches_schedule_phase16, historical_win_loose_draw_ratios, ranking)

# home predicts
matches_schedule_phase16_predicted = run_predict_home(matches_schedule_phase16, historical_results_world_cup)

# away predicts
matches_schedule_phase16_predicted = run_predict_away(matches_schedule_phase16, matches_schedule_phase16_predicted, historical_results_world_cup)

Checking predictions

In [ ]:
matches_schedule_phase16_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team
0,2022-12-03,Netherlands,1.0,0.0,USA
1,2022-12-03,Argentina,2.0,1.0,Australia
2,2022-12-04,France,2.0,1.0,Poland
3,2022-12-04,England,1.0,0.0,Senegal
4,2022-12-05,Japan,1.0,1.0,Croatia
5,2022-12-05,Brazil,3.0,1.0,South Korea
6,2022-12-06,Morocco,1.0,1.0,Spain
7,2022-12-06,Portugal,1.0,1.0,Switzerland


In [ ]:
matches_schedule_phase16_predicted.to_csv('predicts/phase_of_16_predictions.csv', index=False)

Phase of 8

In [ ]:
# ingesting data
data = [
    ['2022-12-09', 'Netherlands', 0, 0, 'Argentina', 'stage of 8'],
    ['2022-12-09', 'Croatia', 0, 0, 'Brazil', 'stage of 8'],
    ['2022-12-10', 'England', 0, 0, 'France', 'stage of 8'],
    ['2022-12-10', 'Morocco', 0, 0, 'Portugal', 'stage of 8']
    ]

# creating pandas DataFrame
matches_schedule_phase8 = pd.DataFrame(data, columns=['date', 'home_team', 'home_score', 'away_score', 'away_team', 'phase'])

# checking data
matches_schedule_phase8

,date,home_team,home_score,away_score,away_team,phase
0,2022-12-09,Netherlands,0,0,Argentina,stage of 8
1,2022-12-09,Croatia,0,0,Brazil,stage of 8
2,2022-12-10,England,0,0,France,stage of 8
3,2022-12-10,Morocco,0,0,Portugal,stage of 8


Enriching data, predicting home and away

In [ ]:
# enrichment
matches_schedule_phase8 = enrich_data(matches_schedule_phase8, historical_win_loose_draw_ratios, ranking)

# home predicts
matches_schedule_phase8_predicted = run_predict_home(matches_schedule_phase8, historical_results_world_cup)

# away predicts
matches_schedule_phase8_predicted = run_predict_away(matches_schedule_phase8, matches_schedule_phase8_predicted, historical_results_world_cup)

Checking predictions

In [ ]:
matches_schedule_phase8_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team
0,2022-12-09,Netherlands,1.0,1.0,Argentina
1,2022-12-09,Croatia,1.0,2.0,Brazil
2,2022-12-10,England,1.0,1.0,France
3,2022-12-10,Morocco,2.0,1.0,Portugal


In [ ]:
matches_schedule_phase8_predicted.to_csv('predicts/phase_of_8_predicts.csv', index=False)

## Semifinal

In [ ]:
# ingesting data
data = [
    ['2022-12-13', 'Argentina', 0, 0, 'Croatia', 'semifinal'],
    ['2022-12-14', 'France', 0, 0, 'Morocco', 'semifinal']
    ]

# creating pandas DataFrame
matches_schedule_semifinals = pd.DataFrame(data, columns=['date', 'home_team', 'home_score', 'away_score', 'away_team', 'phase'])

# checking data
matches_schedule_semifinals

,date,home_team,home_score,away_score,away_team,phase
0,2022-12-13,Argentina,0,0,Croatia,semifinal
1,2022-12-14,France,0,0,Morocco,semifinal


Enriching data, predicting home and away

In [ ]:
# enrichment
matches_schedule_semifinals = enrich_data(matches_schedule_semifinals, historical_win_loose_draw_ratios, ranking)

# home predicts
matches_schedule_semifinals_predicted = run_predict_home(matches_schedule_semifinals, historical_results_world_cup)

# away predicts
matches_schedule_semifinals_predicted = run_predict_away(matches_schedule_semifinals, matches_schedule_semifinals_predicted, historical_results_world_cup)

Checking predictions

In [ ]:
matches_schedule_semifinals_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team
0,2022-12-13,Argentina,1.0,1.0,Croatia
1,2022-12-14,France,2.0,1.0,Morocco


In [ ]:
matches_schedule_semifinals_predicted.to_csv('predicts/semifinals_predicts.csv', index=False)

## Final and Third Place

In [ ]:
# ingesting data
data = [
    ['2022-12-17', 'Croatia', 0, 0, 'Morocco', 'third place'],
    ['2022-12-18', 'Argentina', 0, 0, 'France', 'final']
    ]

# creating pandas DataFrame
matches_schedule_finals = pd.DataFrame(data, columns=['date', 'home_team', 'home_score', 'away_score', 'away_team', 'phase'])

# checking data
matches_schedule_finals

,date,home_team,home_score,away_score,away_team,phase
0,2022-12-17,Croatia,0,0,Morocco,third place
1,2022-12-18,Argentina,0,0,France,final


Enriching data, predicting home and away

In [ ]:
# enrichment
matches_schedule_finals = enrich_data(matches_schedule_finals, historical_win_loose_draw_ratios, ranking)

# home predicts
matches_schedule_finals_predicted = run_predict_home(matches_schedule_finals, historical_results_world_cup)

# away predicts
matches_schedule_finals_predicted = run_predict_away(matches_schedule_finals, matches_schedule_finals_predicted, historical_results_world_cup)

Checking prediction

In [ ]:
matches_schedule_finals_predicted

,date,home_team,home_score_predicted,away_score_predicted,away_team
0,2022-12-17,Croatia,1.0,1.0,Morocco
1,2022-12-18,Argentina,1.0,1.0,France


In [ ]:
matches_schedule_finals_predicted.to_csv('predicts/finals_predicts.csv', index=False)